***
*Project:* Helmholtz Machine on Niche Construction

*Author:* Jingwei Liu, Computer Music Ph.D., UC San Diego
***

# <span style="background-color:darkorange; color:white; padding:2px 6px">Experiment 3_3</span> 

# Real-Time Synthesis (Threading)

*Created:* December 24, 2023

*Updated:* December 24, 2023

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import pyaudio
import wave
import sys
import os
from pathlib import Path
import time
import threading

### An Example of Threading

In [10]:
a = 0
b = 0
def thread1(threadname):
    global a,b
    while a < 10:
        a += 1
        b += 2
        time.sleep(1)

def thread2(threadname):
    while a < 10:
        print(a+b)
        time.sleep(1)

thread1 = threading.Thread(target=thread1, args=("Thread-1", ) )
thread2 = threading.Thread(target=thread2, args=("Thread-2", ) )

thread1.start()
thread2.start()
thread1.join()
thread2.join()

3
6
9
12
15
18
21
24
27


### Functions

In [4]:
txt_folder = Path('Instruments').rglob('*.wav')
Instruments = []
for x in txt_folder:
    basename = os.path.basename(x)
    filename = os.path.splitext(basename)[0]
    Instruments.append(filename)
instruments_sound = {}
for i in range(len(Instruments)):
    instruments_sound[Instruments[i]],fs = librosa.load('Instruments/'+Instruments[i]+'.wav')
instruments_sound

{'clap': array([-5.2706543e-02,  6.8760648e-02, -1.4342493e-01, ...,
         8.6916843e-06, -2.4170677e-06,  9.0632864e-07], dtype=float32),
 'hihat closed': array([ 8.7055489e-02,  3.4345269e-01, -4.2524356e-01, ...,
         5.6425459e-05,  1.7882790e-05, -3.8322993e-05], dtype=float32),
 'hihat open': array([ 9.8636545e-02,  2.8145188e-01, -1.9611624e-01, ...,
        -7.8135054e-07,  1.1714292e-07,  1.0206713e-07], dtype=float32),
 'kick': array([-0.00218478, -0.00930428, -0.01938734, ..., -0.00053879,
        -0.00056686, -0.00048317], dtype=float32),
 'ride': array([-4.1870825e-02,  2.7795248e-02, -1.4001541e-02, ...,
         3.8695944e-05,  6.7816058e-05, -1.5612140e-04], dtype=float32),
 'snare': array([ 7.6284137e-04,  9.2353951e-03,  7.4334239e-04, ...,
        -2.6067326e-05, -1.1731618e-05,  0.0000000e+00], dtype=float32),
 'Tom': array([-1.0707259e-02, -1.9491711e-01, -3.7550956e-01, ...,
         1.3712779e-06,  7.7723962e-07,  1.7710408e-07], dtype=float32),
 'wood': a

In [5]:
fs

22050

In [2]:
def intrument_choice(n_instr):
    # n_instr at most 8
    
    Instruments = [['kick','Tom'],['snare','clap'],['hihat closed','wood'],['ride','hihat open']]
    instr_choice = ['']*n_instr
    
    choice1_group = np.random.choice(4, 4, replace=False, p = [0.4, 0.3, 0.2, 0.1])
    choice_index = np.random.randint(2,size = 4)
    choice1_inst = choice_index[choice1_group]
    choice2_group = np.random.choice(4, 4, replace=False)
    choice2_inst = np.mod(choice_index+1,2)[choice2_group]
    choice = np.array([np.append(choice1_group,choice2_group),np.append(choice1_inst,choice2_inst)])
    
    for i in range(n_instr):
        instr_choice[i] = Instruments[choice[0,i]][choice[1,i]]
            
    return instr_choice

In [3]:
def y_init(division,BPM,fs):
    BS = 60/BPM # beat length in seconds
    len_gen = int(BS * fs * division + 0.5)
    y = np.zeros((len_gen*2,))
    return y,len_gen

In [4]:
def synthesis(data,division,instr_choice,instruments_sound,y,deviation):
    """
    Arguments:
    data -- generated single data point of length n, numpy array of shape (n,1)
    division -- number of beats per instrument track, integer number of 4,5,6
    instr_choice -- based on division and data length, we compute the number of instruments is n_instr = int(n/division+0.5), 
                    instr_choice is a list of chosen instrument names of shape length n_instr
    instruments_sound -- Python disctionary with key: instrument name & value: instrument wave form in floating number farmat [-1,1]
    y -- residual of previous generation for adding on, numpy array of shape (int(60/BPM * fs * division + 0.5)*2, )
    deviation -- deviation from beat grid in samples, a number likely 500-1000
    
    Returns:
    y -- generated audio in floating number farmat [-1,1], numpy array of shape (int(60/BPM * fs * division + 0.5)*2, )
    y_byte -- byte format of first half of y, namely y[:len(y)/2]
    """
    n = len(data)
    n_instr = int(n/division+0.5)
    y_beat = int(len(y)/8 + 0.5)
    
    for i in range(n_instr-1):
        notes = data[i*division:(i+1)*division]
        y_instr = instruments_sound[instr_choice[i]]
        for j in range(division):
            if notes[j] != 0:
                randomize = int(np.random.randn()*deviation)
                st = np.max([0,y_beat*j+randomize])
                y[st:st+len(y_instr)] += y_instr*np.random.rand()

    i = i+1
    notes = data[i*division:]
    k = len(notes)
    y_instr = instruments_sound[instr_choice[i]]
    for j in range(k):
        pos = np.random.choice(division, k, replace=False)
        if notes[j] != 0:
            randomize = int(np.random.randn()*deviation)
            st = np.max([0,y_beat*pos[j]+randomize])
            y[st:st+len(y_instr)] += y_instr*np.random.rand()
    
    y_max = np.max(np.abs(y))
    if y_max > 1:
        y = y/y_max*0.98
    
    # Int16 -- (-32,768 to +32,767)    
    y_int16 = (y[:int(len(y)/2)] * 32768 - 0.5).astype('int16')
    y_byte = y_int16.tobytes()
    
    return y, y_byte

### Function Test

In [5]:
def random_generate(k,n):
    
    u = np.random.rand(k,)
    c = np.random.randint(k, size=(n,1))
    mean = u[c]
    prob = np.random.randn(n,1) + mean
    random_gen = (prob>0.5).astype(int)
    
    return random_gen

In [11]:
n = 15
data = random_generate(5,n)
data

array([[1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0]])

In [28]:
division = 4
n_instr = int(n/division+0.5)
n_instr

4

In [29]:
instr_choice = intrument_choice(n_instr)
instr_choice

['clap', 'hihat closed', 'kick', 'hihat open']

In [70]:
BPM = 120
y,len_gen = y_init(division,BPM,fs)
y

array([0., 0., 0., ..., 0., 0., 0.])

In [71]:
len_gen

44100

In [67]:
deviation = 600
y_out, y_byte = synthesis(data,division,instr_choice,instruments_sound,y,deviation)

In [68]:
y_out

array([-0.0295079 ,  0.03147832, -0.08805773, ...,  0.        ,
        0.        ,  0.        ])

### Real-Time Synthesis

Global variables: y_out,y_byte,y_byte_next,flag,event

In [53]:
# lock
lock = threading.Lock()
lock

<unlocked _thread.lock object at 0x00000238147CB6C0>

In [54]:
lock.acquire() # only run once after release [deadlock]
lock

<locked _thread.lock object at 0x00000238147CB6C0>

In [55]:
lock.release()
lock

<unlocked _thread.lock object at 0x00000238147CB6C0>

In [48]:
event = threading.Event()
event

<threading.Event at 0x2381464cd90: unset>

In [76]:
event.set()
event

<threading.Event at 0x2381464cd90: set>

In [75]:
event.clear()
event

<threading.Event at 0x2381464cd90: unset>

In [18]:
def streaming(threadname):
    global flag
    pya = pyaudio.PyAudio()
    stream = pya.open(format=pya.get_format_from_width(width=2), channels=1, rate=fs, output=True)

    while np.any(y_out!=0):
        stream.write(y_byte)
        flag = 1
        event.set()
        
        stream.write(y_byte_next)
        flag = 2
        event.set()
        
    stream.stop_stream()
    stream.close()
    pya.terminate()

In [19]:
def control(threadname):
    global gen,y_out #, y_byte, y_byte_next, y_out
    while gen < 5:  # gen == True
        data = random_generate(5,n)
        y_out, y_buffer = synthesis(data,division,instr_choice,instruments_sound,y,deviation)
        y.fill(0)
        y[:len_gen] = y_out[len_gen:]
        
        event.wait()
        if flag == 1:
            y_byte = y_buffer
        elif flag == 2:
            y_byte_next = y_buffer
        event.clear()
        gen += 1
    y_out.fill(0)

In [20]:
# Preparation
n = 15
division = 4
n_instr = int(n/division+0.5)
instr_choice = intrument_choice(n_instr)

txt_folder = Path('Instruments').rglob('*.wav')
Instruments = []
for x in txt_folder:
    basename = os.path.basename(x)
    filename = os.path.splitext(basename)[0]
    Instruments.append(filename)
instruments_sound = {}
for i in range(len(Instruments)):
    instruments_sound[Instruments[i]],fs = librosa.load('Instruments/'+Instruments[i]+'.wav')

BPM = 120
deviation = 600

y,len_gen = y_init(division,BPM,fs)
data = random_generate(5,n)
y_out, y_byte = synthesis(data,division,instr_choice,instruments_sound,y,deviation)
y.fill(0)
y[:len_gen] = y_out[len_gen:]
data = random_generate(5,n)
y_out, y_byte_next = synthesis(data,division,instr_choice,instruments_sound,y,deviation)

In [21]:
y_out

array([-0.00066547, -0.00283404, -0.00590529, ...,  0.        ,
        0.        ,  0.        ])

In [ ]:
# Threading
event = threading.Event()
flag = 0
gen = 0
thread1 = threading.Thread(target=streaming, args=("Thread-1", ) )
thread2 = threading.Thread(target=control, args=("Thread-2", ) )

thread1.start()
thread2.start()
thread1.join()
thread2.join()